In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [3]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [4]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [5]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [6]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

In [6]:
naep_wrong_resp.text

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2380\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2010\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 651\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 191"}'

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [7]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [8]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2380\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2010\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 651\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 191"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [9]:
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')

In [11]:
def naep_api_calls(grades: list,
                   baseurl = 'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?',
                   query_part1 = 'type=data&subject=writing',
                   query_part3 = '&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'):
                       
    '''This function calls the NAEP API to get data on writing scores for the grade requested in 2011.
    
    Parameters:
        baseurl (str): default URL for NAEP or similar API
        query_part1 (str): first batch of arguments
        grades (list of int): what grades to get data on
        query_part3 (str): third batch of arguments
    Returns:
        DataFrame with the results'''
    
    dflist = []
    
    for grade in grades:
        naep_query = baseurl + query_part1  + query_part3 + ('&grade=' + str(grade))
        naep_resp = requests.get(naep_query)
        
        #if "System.Exception" in naep_resp.text:
        #    print(f"NAEP results not found for grade {str(grade)}, please try with grade in range of 1-12")
        #else:
        try:
            naep_resp_df = pd.DataFrame(naep_resp.json()['result'])
            dflist.append(naep_resp_df)
        except Exception as e:
            print("Failed to get result from API for grade {} due to error:".format(str(grade)))
            print(e)
        
    combined_result_df = pd.concat(dflist)
    
    return(combined_result_df)


naep_api_calls(grades = [4,8,12])

Failed to get result from API for grade 4 due to error:
Invalid control character at: line 1 column 289 (char 288)


,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0
0,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,1,Male,141.256978,1,0
1,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,2,Female,155.385917,1,0


# 2. Example 2: needs credentials; no wrapper

In [ ]:
# Format example

In [9]:
API_KEY = "Your Key"

In [10]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [11]:
yelp_genjson["businesses"]

[{'id': 'XVGEEIH5rVB2QzW-qywcJw',
  'alias': 'base-camp-cafe-hanover',
  'name': 'Base Camp Cafe',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiEQI7p_JF82eaWa9Xg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/base-camp-cafe-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
  'review_count': 247,
  'categories': [{'alias': 'himalayan', 'title': 'Himalayan/Nepalese'}],
  'rating': 4.4,
  'coordinates': {'latitude': 43.700626, 'longitude': -72.2887803},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '3 Lebanon St',
   'address2': 'Ste 13',
   'address3': '',
   'city': 'Hanover',
   'zip_code': '03755',
   'country': 'US',
   'state': 'NH',
   'display_address': ['3 Lebanon St', 'Ste 13', 'Hanover, NH 03755']},
  'phone': '+16036432007',
  'display_phone': '(603) 643-2007',
  'distance': 196.1397581876442},
 {'id': '5WW4g_LRwau29KyjZGLy

In [18]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'XVGEEIH5rVB2QzW-qywcJw',
 'alias': 'base-camp-cafe-hanover',
 'name': 'Base Camp Cafe',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiEQI7p_JF82eaWa9Xg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/base-camp-cafe-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 238,
 'categories': [{'alias': 'himalayan', 'title': 'Himalayan/Nepalese'}],
 'rating': 4.5,
 'coordinates': {'latitude': 43.700626, 'longitude': -72.2887803},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3 Lebanon St',
  'address2': 'Ste 13',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['3 Lebanon St', 'Ste 13', 'Hanover, NH 03755']},
 'phone': '+16036432007',
 'display_phone': '(603) 643-2007',
 'distance': 196.1397581876442}

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,238,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.5,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758
1,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,500,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160
2,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,19,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.0,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552
3,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,False,https://www.yelp.com/biz/murphy-s-on-the-green...,173,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 43.702, 'longitude': -72.2894949}",[delivery],$$,"{'address1': '11 S Main St', 'address2': '', '...",+16036434075,(603) 643-4075,295.207523
4,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,374,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,244.006059


In [32]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
neBEWvgHNhsXIBImCYx_6A,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,https://www.yelp.com/biz/murphy-s-on-the-green...,$$,+16036434075,(603) 643-4075
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
34j_2nRCVQBTKafJvncZlg,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/rD7w_r...,https://www.yelp.com/biz/pine-restaurant-hanov...,$$$,+16036468000,(603) 646-8000
vMyN7JL5cJExJORgIobbQg,vMyN7JL5cJExJORgIobbQg,tuk-tuk-thai-cuisine-hanover,Tuk Tuk Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/--bUG3...,https://www.yelp.com/biz/tuk-tuk-thai-cuisine-...,$$,+16032779192,(603) 277-9192


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://www.yelp.com/developers/documentation/v3/business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [12]:
# change location
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
# my_location_marlow = "Marlow,NH,03456"
my_location_tpe = "Taipei,Taiwan,03456"
yelp_genquery_tpe = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location_marlow)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_ = requests.get(yelp_genquery_tpe, headers = header)
yelp_genresp_tpe

## then, look at structure of response
yelp_genjson_marlow = yelp_genresp_marlow.json()

## turn JSON into usable data (DF)
yelp_gendf_marlow = pd.DataFrame(yelp_genjson_marlow['businesses'])
#list(yelp_gendf_marlow) # list columns
yelp_gendf_marlow['location1'] = yelp_gendf_marlow.location.apply(lambda loclist: loclist['address1'])
yelp_gendf_marlow[['alias', 'name', 'url', 'review_count', 'rating', 'location1', 'price']]

# Optional: Restrict to cols that are just strings
#yelp_stronly_marlow = [clean_yelp_json(one_b) for one_b in yelp_genjson_marlow['businesses']]
#yelp_stronly_marlow_df = pd.concat(yelp_stronly_marlow)

#yelp_stronly_marlow_df.head(10)

NameError: name 'API_KEY' is not defined

In [30]:
# change location
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location_marlow = "Marlow,NH,03456"
yelp_genquery_marlow = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location_marlow)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_marlow = requests.get(yelp_genquery_marlow, headers = header)
yelp_genresp_marlow

## then, look at structure of response
yelp_genjson_marlow = yelp_genresp_marlow.json()

## turn JSON into usable data (DF)
yelp_gendf_marlow = pd.DataFrame(yelp_genjson_marlow['businesses'])
#list(yelp_gendf_marlow) # list columns
yelp_gendf_marlow['location1'] = yelp_gendf_marlow.location.apply(lambda loclist: loclist['address1'])
yelp_gendf_marlow[['alias', 'name', 'url', 'review_count', 'rating', 'location1', 'price']]

# Optional: Restrict to cols that are just strings
#yelp_stronly_marlow = [clean_yelp_json(one_b) for one_b in yelp_genjson_marlow['businesses']]
#yelp_stronly_marlow_df = pd.concat(yelp_stronly_marlow)

#yelp_stronly_marlow_df.head(10)

<Response [200]>

,alias,name,url,review_count,rating,location1,price
0,mill-village-country-store-stoddard,Mill Village Country Store,https://www.yelp.com/biz/mill-village-country-...,13,4.5,984 Rte 123 N,NaN
1,hilltop-kitchen-harrisville,Hilltop Kitchen,https://www.yelp.com/biz/hilltop-kitchen-harri...,3,5.0,184 Aldworth Manor Rd,NaN
2,auntie-ts-roadside-grill-and-catering-stoddard,Auntie T's Roadside Grill & Catering,https://www.yelp.com/biz/auntie-ts-roadside-gr...,8,4.5,475 Rte 9,NaN
3,kirbys-q-alstead,Kirby's Q,https://www.yelp.com/biz/kirbys-q-alstead?adju...,25,4.5,163 River St,$$
4,washington-general-store-washington,Washington General Store,https://www.yelp.com/biz/washington-general-st...,14,4.0,29 N Main St,$$$
5,macdougalls-tavern-keene,MacDougall's Tavern,https://www.yelp.com/biz/macdougalls-tavern-ke...,5,4.5,630 Park Ave,$
6,lighthouse-westmoreland,Lighthouse,https://www.yelp.com/biz/lighthouse-westmorela...,0,0.0,719 Route 12,NaN
7,pizza-down-under-keene,Pizza Down Under,https://www.yelp.com/biz/pizza-down-under-keen...,20,4.0,800 Park Ave,$
8,guru-food-truck-keene,Guru Food Truck,https://www.yelp.com/biz/guru-food-truck-keene...,1,5.0,None,NaN
9,elm-st-market-keene,Elm St Market,https://www.yelp.com/biz/elm-st-market-keene?a...,8,5.0,130 Elm St,$


In [33]:
# Get one random business
one_biz = yelp_stronly_df.sample()[['name', 'id']]
one_biz

,name,id
Ecdu5qYM09F647Uoez99vg,Ramunto's Brick & Brew Pizzeria,Ecdu5qYM09F647Uoez99vg


In [42]:
url = "https://api.yelp.com/v3/businesses/Ecdu5qYM09F647Uoez99vg/reviews?"
# header = {‘Authorization’: f”Bearer {API_KEY}""}
response = requests.get(url, headers=header)
print(response.text)


{"reviews": [{"id": "UpqByKb8iBIFzYvFmvtt0A", "url": "https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=UpqByKb8iBIFzYvFmvtt0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w", "text": "Best pizza ever from this townie. Even other towns nearby... still can't get me to fully change my mind on how this place has the best sauce, pies, and noms...", "rating": 5, "time_created": "2023-08-08 06:53:00", "user": {"id": "kgAiZU60RefGdjkeRdi0pA", "profile_url": "https://www.yelp.com/user_details?userid=kgAiZU60RefGdjkeRdi0pA", "image_url": "https://s3-media2.fl.yelpcdn.com/photo/lLzLXgaHEtgaPwhPVfs74g/o.jpg", "name": "Mia A."}}, {"id": "Yxj8gWX2isGf0-DVBPq67A", "url": "https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=Yxj8gWX2isGf0-DVBPq67A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w

In [39]:
response.text

'{"reviews": [{"id": "8I8ALslMWvOQmd7AnO4N8w", "url": "https://www.yelp.com/biz/the-shed-restaurant-plainview-plainview?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=8I8ALslMWvOQmd7AnO4N8w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w", "text": "I LOVE THE SHED!\\n\\nAny location doesn\'t matter my experience has blown me away and left me satisfied.\\n\\nThe grilled chicken entree was better than my grilled...", "rating": 5, "time_created": "2023-09-23 14:26:01", "user": {"id": "HOxaVDup_Ctag_npGDdF0A", "profile_url": "https://www.yelp.com/user_details?userid=HOxaVDup_Ctag_npGDdF0A", "image_url": "https://s3-media3.fl.yelpcdn.com/photo/P78IjRv3JfpOcjE0T1dgqw/o.jpg", "name": "Emily B."}}, {"id": "sPyEsPtJz48wyOd3DIrtGA", "url": "https://www.yelp.com/biz/the-shed-restaurant-plainview-plainview?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=sPyEsPtJz48wyOd3DIrtGA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RR

In [40]:
# look at reviews of business with this id
base_url_reviews = f'https://api.yelp.com/v3/businesses/{one_biz.id[0]}/reviews'
yelp_genquery_reviews = (base_url_reviews)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_reviews = requests.get(yelp_genquery_reviews, headers = header)
yelp_genresp_reviews

## then, look at structure of response
yelp_genjson_reviews = yelp_genresp_reviews.json()
yelp_genjson_reviews

<Response [200]>

{'reviews': [{'id': 'UpqByKb8iBIFzYvFmvtt0A',
   'url': 'https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=UpqByKb8iBIFzYvFmvtt0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w',
   'text': "Best pizza ever from this townie. Even other towns nearby... still can't get me to fully change my mind on how this place has the best sauce, pies, and noms...",
   'rating': 5,
   'time_created': '2023-08-08 06:53:00',
   'user': {'id': 'kgAiZU60RefGdjkeRdi0pA',
    'profile_url': 'https://www.yelp.com/user_details?userid=kgAiZU60RefGdjkeRdi0pA',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/lLzLXgaHEtgaPwhPVfs74g/o.jpg',
    'name': 'Mia A.'}},
  {'id': 'Yxj8gWX2isGf0-DVBPq67A',
   'url': 'https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=Yxj8gWX2isGf0-DVBPq67A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&u

In [41]:
yelp_genjson_reviews['reviews']

[{'id': 'UpqByKb8iBIFzYvFmvtt0A',
  'url': 'https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=UpqByKb8iBIFzYvFmvtt0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w',
  'text': "Best pizza ever from this townie. Even other towns nearby... still can't get me to fully change my mind on how this place has the best sauce, pies, and noms...",
  'rating': 5,
  'time_created': '2023-08-08 06:53:00',
  'user': {'id': 'kgAiZU60RefGdjkeRdi0pA',
   'profile_url': 'https://www.yelp.com/user_details?userid=kgAiZU60RefGdjkeRdi0pA',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/lLzLXgaHEtgaPwhPVfs74g/o.jpg',
   'name': 'Mia A.'}},
 {'id': 'Yxj8gWX2isGf0-DVBPq67A',
  'url': 'https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=Yxj8gWX2isGf0-DVBPq67A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_R

In [45]:
yelp_genjson_reviews

{'reviews': [{'id': 'UpqByKb8iBIFzYvFmvtt0A',
   'url': 'https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=UpqByKb8iBIFzYvFmvtt0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w',
   'text': "Best pizza ever from this townie. Even other towns nearby... still can't get me to fully change my mind on how this place has the best sauce, pies, and noms...",
   'rating': 5,
   'time_created': '2023-08-08 06:53:00',
   'user': {'id': 'kgAiZU60RefGdjkeRdi0pA',
    'profile_url': 'https://www.yelp.com/user_details?userid=kgAiZU60RefGdjkeRdi0pA',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/lLzLXgaHEtgaPwhPVfs74g/o.jpg',
    'name': 'Mia A.'}},
  {'id': 'Yxj8gWX2isGf0-DVBPq67A',
   'url': 'https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=Yxj8gWX2isGf0-DVBPq67A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&u

In [18]:
## turn JSON into usable data (DF)
yelp_gendf_reviews = pd.DataFrame(yelp_genjson_reviews['reviews'])
yelp_gendf_reviews

,id,url,text,rating,time_created,user
0,ZKw3pgptzE3aKiKmwFLK1Q,https://www.yelp.com/biz/sushiya-hanover-2?adj...,Just had an phenomenal meal here and look forw...,5,2022-08-25 19:19:55,"{'id': '76gvqovFuTpEoOHysjbY6w', 'profile_url'..."
1,nzGzOK01t_DztjuYGwB4SQ,https://www.yelp.com/biz/sushiya-hanover-2?adj...,This restaurant is now takeout only. The sushi...,3,2022-10-17 18:15:08,"{'id': 'trIbciU6PXfiXr0BYz0DQw', 'profile_url'..."
2,xfBpLVOxGF9UZAoe8wQpBw,https://www.yelp.com/biz/sushiya-hanover-2?adj...,Ordered takeout for 6 rolls. All of them taste...,4,2021-10-14 19:20:23,"{'id': 'Imrkj1Ywx5RVNDmDh9AFcw', 'profile_url'..."
